# Técnicas Avançadas de **Captura e Tratamento** de Dados

---
## Prof. Bernardo Alves Furtado
---
### MBA em Big Data, Business Analytics e Gestão de Negócios. @**IDP**
---
3 a 21 agosto  -- 21 horas/aula

## Fonte: Valor 16/8/2021. Manchete:
# Pessimismo da comunidade financeira no Twitter aumenta

![Finwit](https://github.com/BAFurtado/MBA_IDP_CapturaTratamento/blob/main/data/Screenshot%20from%202021-08-17%2013-11-05.png?raw=True)
### Ilustração de utilização de prática
### Matéria original: https://valor.globo.com/financas/noticia/2021/08/16/pessimismo-da-comunidade-financeira-no-twitter-aumenta.ghtml


In [1]:
import requests
url = 'https://queridodiario.ok.org.br/api/'
endpoint = 'gazettes/'

params = {'since': '2020-12-15',
          'until': '2020-12-31',
          'keywords': ['pandemia']}

r = requests.get(f'{url}{endpoint}', params=params).json()
r

{'total_gazettes': 82,
 'gazettes': [{'territory_id': '2927408',
   'date': '2020-12-31',
   'url': 'https://querido-diario.nyc3.cdn.digitaloceanspaces.com/2927408/2020-12-31/b13e9cc325502b91376f8fb1a7577a22f7bff477.pdf',
   'territory_name': 'Salvador',
   'state_code': 'BA',
   'highlight_texts': ['à Pandemia do Covid-19 - SEMOB Ações Realizadas Percentual Inter-Regionais  100,00  100.000 \n\n26.122.0002.263019 Enfrentamento à Pandemia do Covid-19'],
   'is_extra_edition': False,
   'file_raw_txt': 'https://querido-diario.nyc3.cdn.digitaloceanspaces.com/2927408/2020-12-31/b13e9cc325502b91376f8fb1a7577a22f7bff477.txt'},
  {'territory_id': '1721000',
   'date': '2020-12-31',
   'url': 'https://querido-diario.nyc3.cdn.digitaloceanspaces.com/1721000/2020-12-31/0d0b3226bebf787a3030ccefc9ef7561eb653eac.pdf',
   'territory_name': 'Palmas',
   'state_code': 'TO',
   'highlight_texts': ['lenta, tendo \n\nque o Ministério da Economia projeta a volta da fase pré-pandemia em 2022.'],
   'is_extr

In [2]:
print(r.keys())

dict_keys(['total_gazettes', 'gazettes'])


In [3]:
print(type(r['gazettes']))

<class 'list'>


In [4]:
print(type(r['gazettes'][0]))

<class 'dict'>


In [5]:
print(r['gazettes'][0].keys())

dict_keys(['territory_id', 'date', 'url', 'territory_name', 'state_code', 'highlight_texts', 'is_extra_edition', 'file_raw_txt'])


In [6]:
for gazette in r['gazettes']:
    try:
        texto = gazette['file_raw_txt']
    except KeyError:
        pass

texto


'https://querido-diario.nyc3.cdn.digitaloceanspaces.com/4205407/2020-12-30/e83f5e1b581d6482ff83d834937fdae741e3a7d4.txt'

In [7]:
textos = list()
for gazette in r['gazettes']:
    try:
        textos.append(gazette['file_raw_txt'])
    except KeyError:
        pass
textos

['https://querido-diario.nyc3.cdn.digitaloceanspaces.com/2927408/2020-12-31/b13e9cc325502b91376f8fb1a7577a22f7bff477.txt',
 'https://querido-diario.nyc3.cdn.digitaloceanspaces.com/1721000/2020-12-31/0d0b3226bebf787a3030ccefc9ef7561eb653eac.txt',
 'https://querido-diario.nyc3.cdn.digitaloceanspaces.com/3304557/2020-12-31/725c7257e6a95847e9771e46b666b3c55878d3bc.txt',
 'https://querido-diario.nyc3.cdn.digitaloceanspaces.com/2507507/2020-12-31/08bbfaeb37b3136770e2fd696a78aec0dfb27009.txt',
 'https://querido-diario.nyc3.cdn.digitaloceanspaces.com/2704302/2020-12-30/6880e353c91313fbc11663d356385f218727a367.txt',
 'https://querido-diario.nyc3.cdn.digitaloceanspaces.com/2704302/2020-12-30/98145868e181afac62524af694ef54f944915ba8.txt',
 'https://querido-diario.nyc3.cdn.digitaloceanspaces.com/2211001/2020-12-30/ec432a7f330442cd32556028fc668f3ac24258c3.txt',
 'https://querido-diario.nyc3.cdn.digitaloceanspaces.com/4205407/2020-12-30/e83f5e1b581d6482ff83d834937fdae741e3a7d4.txt']

---
# Reforçando Captura $+$ Base de Dados
# Exercício II API $+$ SQL



---
1. Seguindo exatamente os passos que utilizamos para ilustrar SQL, vamos simplificar o processo e
criar uma base que atenda os requisitos desta captura de dados de tweets.

2. Conseguem me contar quais passos (em geral) foram esses?

In [8]:
print('')

1. Vamos começar **CRIANDO** a função de conexão com uma base nova. E o nome da nova base.

In [9]:
import sqlite3

2. Configuração de dados básicos.

In [10]:
import os

# Endereço no qual irá guardar a database
tweet_db = 'data/my_first_tweet_database.db'
if not os.path.exists('data'):
    os.mkdir('data')


In [11]:
# Relembrando dados básicos
from getpass import getpass
bearer_token = getpass()

In [12]:
search_url = "https://api.twitter.com/2/tweets/search/recent"

twitter_handle = 'folha'
query_params = {'query': f'(from:{twitter_handle})',
                'tweet.fields': 'author_id'}

In [13]:
def bearer_oauth(bear):
    """
    Method required by bearer token authentication.
    """

    bear.headers["Authorization"] = f"Bearer {bearer_token}"
    bear.headers["User-Agent"] = "v2RecentSearchPython"
    return bear


def connect_to_endpoint(url, params):
    response = requests.get(url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()


3. Função genérica de conexão

In [14]:
def connect_db(database):
    conn = sqlite3.connect(database)
    print(f'Conexão realizada com sucesso.')
    return conn

4. Na sequência, vamos adaptar os comandos **SQL** para a CRIAÇÃO DE TABELAS na base nova.

 * Como exemplo, vamos fazer só uma tabela. Sugiro realizarem de forma mais completa posteriormente.

In [15]:
sql_create_table = """CREATE TABLE IF NOT EXISTS tweets (
                                id integer PRIMARY KEY,
                                texto text NOT NULL,
                                author_id NOT NULL
                            );"""

5. Com a função de conexão, o nome da base e a instrução para criação da tabela, vamos DE FACTO, criar a tabela.
 * Simplesmente passa o nome da base, executa o comando de criar tabela e *commit* (salva localmente).

In [16]:
# Rode somente uma vez para criar a table.
with connect_db(tweet_db) as con:
    con.execute(sql_create_table)
    con.commit()

Conexão realizada com sucesso.


6. Agora a parte distinta do exemplo anterior. Temos que inserir o processo de ATUALIZAÇÃO da base SQL ao processo de consulta da API!
7. Vamos utilizar a mesma função `main` anterior, porém, **alterando**:
 1. No momento de atualização da base.
 2. E na consulta ao valor máximo presente na base.

In [17]:
def insere_tweet(conn, tweet):
    sql = ''' INSERT INTO tweets(author_id,id,texto)
              VALUES(?,?,?) '''
    cur = conn.cursor()
    # Note que 'tweet' precisa ser recebido como uma lista com sequência de id, author_id e texto
    cur.executemany(sql, tweet)
    conn.commit()
    return cur.lastrowid

In [18]:
def check_since_id(conn):
    # Note que a condição abaixo é necessária para verificar se há algum item com id na base
    sql = '''SELECT MAX(id) FROM tweets;'''
    cur = conn.cursor()
    cur.execute(sql)
    return cur.fetchone()[0]

In [19]:
def main(database):
    # Abre a conexão com SQL
    with connect_db(database) as conn:
        next_token = True
        while next_token:
            # Verificando se o tweet mais recente já está na base
            max_value = check_since_id(con)
            if max_value:
                query_params['since_id'] = max_value
            json_response = connect_to_endpoint(search_url, query_params)
            # Verificando se é necessária mais de uma conexão, ou se já é a última ou única.
            if 'next_token' in json_response['meta']:
                next_token = json_response['meta']['next_token']
                print(next_token)
                query_params['next_token'] = next_token
            else:
                next_token = False
            if 'data' in json_response:
                # Novamente. json_response['data'] é uma lista.
                # Vamos precisar dos dicionários que estão dentro da lista.
                tweets = [list(x.values()) for x in json_response['data']]
                insere_tweet(conn, tweets)

Chama a função com a database

In [20]:
main(tweet_db)

Conexão realizada com sucesso.
200
b26v89c19zqg8o3fpdm9f1vx8vyprflmajtyhulpr1yf1
200


In [53]:
# Examinando a base
def collect_basics(conn):
    # Note que a condição abaixo é necessária para verificar se há algum item com id na base
    sql = '''SELECT * FROM tweets LIMIT 5;'''
    cur = conn.cursor()
    cur.execute(sql)
    return cur.fetchall()

In [55]:
with connect_db(tweet_db) as conn:
    res = collect_basics(conn)

for each in res:
    print(each)

Conexão realizada com sucesso.
(1427648644916944903, "PK diz que tem Seu Jorge como inspiração: 'Não me considero só do rap' https://t.co/03bdtYJYV2", '14594813')
(1427648963625242627, 'Em meio à pandemia de Covid, vendas de bicicleta sobem 34% no semestre https://t.co/QvDQbdVg2D', '14594813')
(1427649852847046663, "Afegãos têm motivos para ter medo do Taleban, diz autor de 'O Caçador de Pipas' https://t.co/aOmGvHKwIk", '14594813')
(1427650951817711616, 'Crise hídrica pode encarecer conta de luz, alerta Aneel https://t.co/wwqtVqeWj3', '14594813')
(1427651429980852233, 'Milton Ribeiro cita calote do Fies para justificar fala de que universidade é para poucos https://t.co/c0VnDbSr1u', '14594813')


# Aperitivo: análise textual
## Vamos começar com os dados textuais de notícias que obtivemos com a API do Twitter

1. Obtendo a base no pandas
2. Note que usamos `sep=';'` na hora de salvar. Temos que especificá-lo na hora de ler o arquivo.

In [21]:
import pandas as pd

In [22]:
address = 'https://github.com/BAFurtado/MBA_IDP_CapturaTratamento/blob/main/data/base_tweets_read.csv'
base_texto = pd.read_csv(f'{address}?raw=True', sep=';')

3. Como sempre, vamos investigar os aspectos gerais da base.

In [23]:
base_texto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1692 entries, 0 to 1691
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  1692 non-null   int64 
 1   id         1692 non-null   int64 
 2   text       1692 non-null   object
dtypes: int64(2), object(1)
memory usage: 39.8+ KB


In [24]:
base_texto.describe()

,author_id,id
count,1692.0,1.692000e+03
mean,14594813.0,1.419468e+18
std,0.0,7.683511e+14
min,14594813.0,1.418223e+18
25%,14594813.0,1.418748e+18
50%,14594813.0,1.419486e+18
75%,14594813.0,1.420155e+18
max,14594813.0,1.420752e+18


4. Conferindo se não há linhas duplicadas. como `duplicated` retorna False ou True para as linhas.
5. E sabemos que True em python = 1 e False = 0. Se a soma é zero, então...

In [25]:
sum(base_texto.duplicated())

0

6. Vamos agora filtrar os tweets para conter apenas as palavras inicialmente.
7. Lembrem.se que o comando `.str` acessa dados das colunas do pandas e disponibiliza as funções de **str**
8. Vamos detalhar mais a questão do <span style="color:red">filtro</span> mais tarde.

In [26]:
base_texto['words'] = base_texto.text.str.strip().str.split('[\W_]+')

9. Vejam como ficou:

In [27]:
base_texto.head()

,author_id,id,text,words
0,14594813,1420751875444416535,Todas as pessoas que já passaram pela ginástic...,"[Todas, as, pessoas, que, já, passaram, pela, ..."
1,14594813,1420751478524764167,"Rede D'Or compra hospital em Feira de Santana,...","[Rede, D, Or, compra, hospital, em, Feira, de,..."
2,14594813,1420750976470818827,"Bolsonaro chama imprensa para live, sem direit...","[Bolsonaro, chama, imprensa, para, live, sem, ..."
3,14594813,1420749210438144006,Nubank aumentará o limite do cartão de quase 9...,"[Nubank, aumentará, o, limite, do, cartão, de,..."
4,14594813,1420748456553947150,"Saiba o que é fibrilação atrial, problema card...","[Saiba, o, que, é, fibrilação, atrial, problem..."


10. Verificando somente uma linha.

In [28]:
base_texto.loc[0]

author_id                                             14594813
id                                         1420751875444416535
text         Todas as pessoas que já passaram pela ginástic...
words        [Todas, as, pessoas, que, já, passaram, pela, ...
Name: 0, dtype: object

11. Dado que transformamos em lista de words, podemos analisar todo o conjunto de palavras de uma vez.
12. Alternativamente, analisaremos cada tweet...

# Tip
### flat_list = [item for sublist in original for item in sublist]

In [29]:
tweets_words = [item for sublist in base_texto.words for item in sublist]
tweets_words

['Todas',
 'as',
 'pessoas',
 'que',
 'já',
 'passaram',
 'pela',
 'ginástica',
 'feminina',
 'do',
 'Brasil',
 'se',
 'veem',
 'nessa',
 'medalha',
 'diz',
 'Rebeca',
 'Andrade',
 'após',
 'ganhar',
 'a',
 'prata',
 'em',
 'Tóquio',
 'JogosOlimpicos',
 'Tokyo2020',
 'https',
 't',
 'co',
 'CfRg6aSQE9',
 'https',
 't',
 'co',
 'pU7dqJZI5U',
 'Rede',
 'D',
 'Or',
 'compra',
 'hospital',
 'em',
 'Feira',
 'de',
 'Santana',
 'na',
 'Bahia',
 'https',
 't',
 'co',
 'I1fwiVoE1e',
 'Bolsonaro',
 'chama',
 'imprensa',
 'para',
 'live',
 'sem',
 'direito',
 'a',
 'perguntas',
 'sobre',
 'suposta',
 'fraude',
 'nas',
 'eleições',
 'https',
 't',
 'co',
 'mLyuThmbFQ',
 'Nubank',
 'aumentará',
 'o',
 'limite',
 'do',
 'cartão',
 'de',
 'quase',
 '90',
 'dos',
 'clientes',
 'https',
 't',
 'co',
 'n57pDZxiMb',
 'Saiba',
 'o',
 'que',
 'é',
 'fibrilação',
 'atrial',
 'problema',
 'cardíaco',
 'comum',
 'e',
 'fácil',
 'de',
 'ignorar',
 'https',
 't',
 'co',
 'GQUqTMxfLW',
 '',
 'Gênesis',
 'Labão'

13. Com a lista completa de palavras, podemos verificar as mais comuns, por exemplo.

In [30]:
import collections

counter=collections.Counter(tweets_words)
counter

Counter({'Todas': 3,
         'as': 69,
         'pessoas': 24,
         'que': 396,
         'já': 37,
         'passaram': 1,
         'pela': 50,
         'ginástica': 27,
         'feminina': 24,
         'do': 510,
         'Brasil': 158,
         'se': 117,
         'veem': 7,
         'nessa': 2,
         'medalha': 74,
         'diz': 133,
         'Rebeca': 16,
         'Andrade': 10,
         'após': 70,
         'ganhar': 5,
         'a': 687,
         'prata': 26,
         'em': 514,
         'Tóquio': 101,
         'JogosOlimpicos': 125,
         'Tokyo2020': 144,
         'https': 1847,
         't': 1847,
         'co': 1844,
         'CfRg6aSQE9': 1,
         'pU7dqJZI5U': 1,
         'Rede': 4,
         'D': 5,
         'Or': 1,
         'compra': 4,
         'hospital': 2,
         'Feira': 1,
         'de': 1369,
         'Santana': 1,
         'na': 336,
         'Bahia': 8,
         'I1fwiVoE1e': 1,
         'Bolsonaro': 129,
         'chama': 1,
         'imprensa

In [31]:
counter.most_common(10)

[('https', 1847),
 ('t', 1847),
 ('co', 1844),
 ('de', 1369),
 ('e', 744),
 ('a', 687),
 ('em', 514),
 ('do', 510),
 ('da', 475),
 ('que', 396)]

In [32]:
counter.most_common(50)

[('https', 1847),
 ('t', 1847),
 ('co', 1844),
 ('de', 1369),
 ('e', 744),
 ('a', 687),
 ('em', 514),
 ('do', 510),
 ('da', 475),
 ('que', 396),
 ('no', 387),
 ('para', 367),
 ('o', 364),
 ('na', 336),
 ('com', 299),
 ('mais', 206),
 ('é', 195),
 ('por', 194),
 ('', 163),
 ('Brasil', 158),
 ('Olimpíadas', 154),
 ('não', 148),
 ('nas', 145),
 ('Tokyo2020', 144),
 ('diz', 133),
 ('Bolsonaro', 129),
 ('A', 126),
 ('JogosOlimpicos', 125),
 ('dos', 122),
 ('se', 117),
 ('RT', 115),
 ('um', 115),
 ('das', 109),
 ('O', 106),
 ('Tóquio', 101),
 ('ao', 99),
 ('sobre', 93),
 ('os', 93),
 ('contra', 91),
 ('como', 82),
 ('foi', 81),
 ('final', 79),
 ('à', 78),
 ('Leia', 75),
 ('medalha', 74),
 ('uma', 74),
 ('Reuters', 72),
 ('após', 70),
 ('as', 69),
 ('nos', 63)]

14. A informação não é assim tão útil, dado que muitas palavras são as chamadas **stop words**
15. Vamos verificar quantas palavras únicas os tweets contêm e também testar se algumas palavras estão presentes na lista.

In [33]:
palavras_teste = ['medalha', 'CPI', 'frio', 'calor', 'Bolsonaro', 'Lula', 'via', 'hoje', 'futuro']

In [34]:
for word in palavras_teste:
    try:
        print(f"{word}: {counter[word]}")
    except KeyError:
        print(f'{word} não está na amostra')

medalha: 74
CPI: 11
frio: 13
calor: 2
Bolsonaro: 129
Lula: 11
via: 5
hoje: 14
futuro: 5


16. Bem, conseguimos alguma (vaga) ideia de relevância de tema, por exemplo, uma indicação que talvez tenhamos
 <span style="color:blue">frio </span> mais à frente... ou que talvez hoje seja mais relevante que o futuro,
do ponto de vista jornalístico?


# Noções de Natural Language

In [35]:
!pip install nltk

In [36]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/furtado/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Filtering stop words

In [37]:
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/furtado/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [38]:
base_texto['tokens'] = base_texto.text.apply(nltk.word_tokenize, language='portuguese')
base_texto.tokens.head()

0    [Todas, as, pessoas, que, já, passaram, pela, ...
1    [Rede, D'Or, compra, hospital, em, Feira, de, ...
2    [Bolsonaro, chama, imprensa, para, live, ,, se...
3    [Nubank, aumentará, o, limite, do, cartão, de,...
4    [Saiba, o, que, é, fibrilação, atrial, ,, prob...
Name: tokens, dtype: object

In [39]:
base_texto.tokens = base_texto.tokens.apply(lambda w: [x.lower() for x in w if x.lower() not in stopwords])
base_texto.loc[0]

author_id                                             14594813
id                                         1420751875444416535
text         Todas as pessoas que já passaram pela ginástic...
words        [Todas, as, pessoas, que, já, passaram, pela, ...
tokens       [todas, pessoas, passaram, ginástica, feminina...
Name: 0, dtype: object

In [40]:
base_texto.tokens.head()


0    [todas, pessoas, passaram, ginástica, feminina...
1    [rede, d'or, compra, hospital, feira, santana,...
2    [bolsonaro, chama, imprensa, live, ,, direito,...
3    [nubank, aumentará, limite, cartão, quase, 90,...
4    [saiba, fibrilação, atrial, ,, problema, cardí...
Name: tokens, dtype: object

In [41]:
fd = nltk.FreqDist(base_texto.tokens.apply(lambda w: w[0]))
fd

FreqDist({'rt': 115, 'brasil': 27, 'bolsonaro': 21, 'após': 18, 'governo': 12, 'folha': 12, 'pandemia': 12, 'seleção': 10, 'justiça': 10, 'ministério': 9, ...})

In [42]:
counter=collections.Counter(base_texto.tokens.apply(lambda w: w[0]))
counter.most_common(40)

[('rt', 115),
 ('brasil', 27),
 ('bolsonaro', 21),
 ('após', 18),
 ('governo', 12),
 ('folha', 12),
 ('pandemia', 12),
 ('seleção', 10),
 ('justiça', 10),
 ('ministério', 9),
 ('rayssa', 8),
 ('``', 7),
 ('mario', 7),
 ('bom', 7),
 ('arthur', 7),
 ('mortes', 7),
 ('veja', 7),
 ('skatista', 7),
 ('@', 7),
 ('saiba', 6),
 ('rebeca', 6),
 ('italo', 6),
 ('entenda', 6),
 ('1921', 6),
 ('hélio', 6),
 ('guedes', 6),
 ('#', 6),
 ('medina', 6),
 ('ifer', 6),
 ('pf', 5),
 ('.', 5),
 ('deu', 5),
 ('primeira', 5),
 ('paulo', 5),
 ('brasileiro', 5),
 ('ouro', 5),
 ('polícia', 5),
 ('charge', 5),
 ('japonês', 5),
 ('fim', 5)]

---
### Noções de
# Regular Expressions -- REGEX
## `import re`

"Regular expressions (called REs, or regexes, or regex patterns) are essentially a tiny, highly specialized programming
language embedded inside Python and made available through the re module. Using this little language,
**you specify the rules for the set of possible strings that you want to match**; this set might contain English
sentences, or e-mail addresses, or TeX commands, or anything you like. You can then ask questions such as
“Does this string match the pattern?”, or “Is there a match for the pattern anywhere in this string?”.
You can also use REs to modify a string or to split it apart in various ways."

source: https://docs.python.org/3/howto/regex.html

## Em partes
1. Linguagem específica, especialista, para <span style="color:red">filtrar</span> texto em grandes porções de texto.
2. Serve para extrair, por exemplo: CPFs, telefones, e-mails, links, etc.
3. Você especifica as REGRAS.
### Não é simples!
4. Depois aplica as regras ao texto e pergunta quais porções satisfazem a regra.
5. Pode **retornar** as porções que satisfazem a regra, ou substituí-las.

* É difícil por que é necessário identificar a função de vários destes "metacharacters" em sequência

# . ^ $ * + ? { } [ ] \ | ( )
---

1. Vamos começar pelo básico. Localizar uma palavra específica.

In [56]:
import re

pattern = 'gmail'
texto = 'Este é um texto que vamos utilizar para testar se os itens que estamos testando estão aqui dentro deste texto ou não, ' \
        'como disse esse texto pode conter endereços de email do tipo furtadobb@gmail.com, paginas da internet, tais como' \
        'htpps://github.com/bafurtado/PolicySpace2'

re.search(pattern, texto)

<re.Match object; span=(189, 194), match='gmail'>

2. O resultado é a posição do primeiro item encontrado que corresponde ao padrão, à regra. Vamos conferir

In [64]:
print(texto[189:194])

gmail


3. Pronto, com isso, já conseguimos localizar palavras específicas.
4. Vamos automatizar o processo, usando o objeto python retornado pela `re.search()`

In [65]:
resultado = re.search(pattern, texto)
resultado.start()

189

In [66]:
print(texto[resultado.start():resultado.end()])

gmail


---
5. **[ ]** servem para classes de palavras dentro dos colchetes.
6. por exemplo [abc] vai localizar todos **as** os **bs** e os **cs** do texto.

*Na verdade, sem o quantifier que veremos em seguida, o processo identifica apenas o primeiro dos itens.*

In [ ]:
pattern = '[abc]'
re.search(pattern, texto)

8. Conferindo, posição do primeiro a do texto

In [ ]:
print(texto[21])

9. Note que isso é diferente de buscar pela expressão 'abc'.
---

# *
9. Vamos introduzir o asterisco: **\*** primeiro quantifier.
10. O **\*** match <span style="color:red">qualquer número de vezes</span> a expressão anterior.
11. Por exemplo 'ca*t' vai encontrar o que?
12. Pausa.

In [ ]:
print('')

13. Passo-a-passo. o 'c' sempre é necessário. 'a' está antes do **\***, então pode estar incluída 0, 1, 2, 3 ... vezes. O 't' precisa estar no final.

In [68]:
testes = ['maria', 'nina', 'combat', 'ct', 'cat', 'caaat', 'cast']

In [69]:
pattern = 'ca*t'
for teste in testes:
    print(f'{teste}: {re.search(pattern, teste)}')


maria: None
nina: None
combat: None
ct: <re.Match object; span=(0, 2), match='ct'>
cat: <re.Match object; span=(0, 3), match='cat'>
caaat: <re.Match object; span=(0, 5), match='caaat'>
cast: None


### Exercício

1. E a expressão: 'a[bcd]*b' vai encontrar o que?

In [ ]:
print('')

R. Qualquer palavra que comece com 'a', termine com 'b' e tenha nenhuma letra ou qualquer número de 'b's, 'c's ou 'd's.

In [70]:
testes = ['adccbb', 'ab', 'axcdb', 'abcxb', 'cat', 'cast']
pattern = 'a[bcd]*b'
for teste in testes:
    print(f'{teste}: {re.search(pattern, teste)}')

adccbb: <re.Match object; span=(0, 6), match='adccbb'>
ab: <re.Match object; span=(0, 2), match='ab'>
axcdb: None
abcxb: <re.Match object; span=(0, 2), match='ab'>
cat: None
cast: None


---
# $$+$$

14. O $$+$$, por sua vez vai encontrar todas as referências com **pelo menos uma** menção, ou mais
15. 'ca+t', por exemplo vai buscar 'cat', 'caaat', 'caaaaaaat', mas não 'ct'.

---
# ?
16. ? significa uma ou nenhuma vez. Pode ter, ou não ter. Por exemplo sócio-econômico ou socioeconomico, podemos buscar por
*'s?cio?econ?mico'*.

In [83]:
pattern = 's[oó]?cio-?econ[oô]?mica'
texto1 = 'A conjuntura sócio-econômica.'
texto2 = 'A conjuntura socioeconômica.'

print(re.search(pattern, texto1))
print(re.search(pattern, texto2))

<re.Match object; span=(13, 28), match='sócio-econômica'>
<re.Match object; span=(13, 27), match='socioeconômica'>


In [84]:
pattern = 's[oó]+cio-?econ[oô]+mica'
texto1 = 'A conjuntura sócio-econômica.'
texto2 = 'A conjuntura socioeconômica.'

print(re.search(pattern, texto1))
print(re.search(pattern, texto2))

<re.Match object; span=(13, 28), match='sócio-econômica'>
<re.Match object; span=(13, 27), match='socioeconômica'>


---
# {}
17. Aproveitando o exemplo para introduzir {} que indica um quantifier específico

In [85]:
pattern = 's[oó]{1}cio-?econ[oô]{1}mica'
texto1 = 'A conjuntura sócio-econômica.'
texto2 = 'A conjuntura socioeconômica.'
texto3 = 'A conjuntura sóocioeconômica.'

print(re.search(pattern, texto1))
print(re.search(pattern, texto2))
print(re.search(pattern, texto3))

<re.Match object; span=(13, 28), match='sócio-econômica'>
<re.Match object; span=(13, 27), match='socioeconômica'>
None


---
# |
17. De novo, aproveitando o exemplo para introduzir | que indica ou [a|b]
18. Reescrevendo exemplo

In [86]:
pattern = 's[o|ó]cio-?econ[o|ô]mica'
texto1 = 'A conjuntura sócio-economica.'
texto2 = 'A conjuntura socioeconômica.'
texto3 = 'A conjuntura sóocioeconômica.'

print(re.search(pattern, texto1))
print(re.search(pattern, texto2))
print(re.search(pattern, texto3))

<re.Match object; span=(13, 28), match='sócio-economica'>
<re.Match object; span=(13, 27), match='socioeconômica'>
None


---
# {}

19. Para facilitar, os parênteses são utilizados para m (mínimo) e n (máximo) de repetições **da expressão anterior**.
20. 'a{1,3}' significa que a expressão anterior, no caso o 'a', deve ter no mínimo uma vez e no máximo 3.21Portanto, a equivalência abaixo é válida

# *   =   {0, }
# +   =   {1, }
# ?   =   {0, 1}

## exatamente 3 vezes = {3}

---

## Outros detalhes.

21. Para buscar todas as letras minúsculas, pode utilizar o hífen para uma sequência. Por exemplo '[a-z]' serve como regra
para todas as letras minúsculas, como '[0-9]' para todos os dígitos.
22. '[a-zA-Z]', portanto captura tudo que for letra.

# ^

^ é utilizado **NO INÍCIO** de uma sequência como exclusão
23. '[^0-9]' excluirá toda a informação numérica da busca.

---
# Extraindo informação com `re`

## Vimos o processo de 'search'. Mas, também podemos utilizar **re** para extração de informação. Captura.

# ()

24. Se os parênteses fecham uma parte da expressão, apenas aquela expressão será retornada.
25. Ou seja, quando identificamos algum elemento de interesse, podemos capturar apenas uma porção da
informação próxima aquele elemento encontrado. Por exemplo:

In [87]:
texto3 = 'nome: Bernardo Alves Furtado \n nome: Claudiomar'
pattern = 'nome:{1}(.*)'

re.findall(pattern, texto3)

[' Bernardo Alves Furtado ', ' Claudiomar']

26. Agora, começamos a complicar. Como previsto.
27. Capturando o e-mail no texto inicial.

In [88]:
pattern = '([\w]+@[\w]+\.[\w]+)'
re.findall(pattern, texto)

['furtadobb@gmail.com']

### Passo-a-passo
1. O '(' começa o processo de retornar o que for encontrado dentro, até o próximo ')'.
2. [\w] refere-se a qualquer uma das alternativas entre [a-zA-Z0-9], ou seja a palavra antes do **@**
3. Pelo menos uma letra é necessária. Portanto [\w]+
4. Em seguida, o @, literalmente.
5. Depois outra palavra, após o arroba. Mesma coisa: [\w]+.
6. Depois um ponto. Como o ponto é um caractere especial, precisa ir com o '\\' para ser considerado ponto.
7. Depois, novamente outra palavra [\w]+ e retorna.

### Vamos agora realizar o mesmo exercício, porém com a base de tweets que utilizamos. Vamos extrar todos os links contidos nos tweets.

1. Relendo a base

In [89]:
import pandas as pd

address = 'https://github.com/BAFurtado/MBA_IDP_CapturaTratamento/blob/main/data/base_tweets_read.csv'
base_texto = pd.read_csv(f'{address}?raw=True', sep=';')

base_texto.text.head()

0    Todas as pessoas que já passaram pela ginástic...
1    Rede D'Or compra hospital em Feira de Santana,...
2    Bolsonaro chama imprensa para live, sem direit...
3    Nubank aumentará o limite do cartão de quase 9...
4    Saiba o que é fibrilação atrial, problema card...
Name: text, dtype: object

In [90]:
base_texto.text.loc[0]

'Todas as pessoas que já passaram pela ginástica feminina do Brasil se veem nessa medalha, diz Rebeca Andrade, após ganhar a prata em Tóquio #JogosOlimpicos #Tokyo2020 \n\nhttps://t.co/CfRg6aSQE9 https://t.co/pU7dqJZI5U'

2. Nosso padrão, sempre vai conter 'https://'

In [91]:
pattern = '(https?://[\w./]+)'
re.findall(pattern, base_texto.text.loc[0])

['https://t.co/CfRg6aSQE9', 'https://t.co/pU7dqJZI5U']

3. Na verdade, no teste, dois links estão disponíveis.
4. O primeiro parece ser o link para a notícia e o segundo é o link do próprio tweet.
5. Vamos investigar sempre os primeiros. Podemos usar, `search()` e capturar o primeiro group, com `.group(0)`

In [92]:
pattern = '(https://[\w./]+)'
re.search(pattern, base_texto.text.loc[0]).group(0)

'https://t.co/CfRg6aSQE9'

6. Para fazer isso para a base toda, vamos utilizar o `re.compile` para ficar mais rápido e utilizar com apply.
7. Confiram que dá no mesmo:

In [93]:
p = re.compile(pattern)
re.search(p, base_texto.text.loc[0]).group(0)

'https://t.co/CfRg6aSQE9'

In [94]:
# base_texto['links'] = base_texto.text.apply(lambda t: re.search(p, t).group(0))

# O apply simples acima, não funciona quando não há match. Por isso temos que implementar uma condicional.
# Quando não houver, match, add None.

base_texto['links'] = base_texto.text.apply(lambda t: re.search(p, t).group(0) if re.search(p, t) else None)
base_texto.links.head()

0    https://t.co/CfRg6aSQE9
1    https://t.co/I1fwiVoE1e
2    https://t.co/mLyuThmbFQ
3    https://t.co/n57pDZxiMb
4    https://t.co/GQUqTMxfLW
Name: links, dtype: object

In [95]:
base_texto.links.tail()

1687    https://t.co/JkthCh42Rv
1688    https://t.co/aZVedrdkMl
1689    https://t.co/Fnkor61377
1690    https://t.co/PQzWVYjlg0
1691    https://t.co/hCxE7yVfgr
Name: links, dtype: object

In [ ]:
base_texto.info()

# Exercícios

1. Para testes automatizados, com explicação detalhada. Porém, só para o primeiro elemento: *match*, mas não *findall*:

https://regex101.com/r/EwvuHA/47

---
# Captura Massiva

## dados.gov

## Ilustração

## *Exemplo de identificação, localização e extração de links de modo a automatizar download (captura) para posterior tratamento*
---
1. No site https://dados.gov.br/organization verificamos 198 organizações.
2. Vamos verificar dados do DIÁRIO OFICIAL. Relevante para análise políticas públicas, mas
também de interesse para empresas (licitações), informações (contratos, concursos), regulação, legislação.
3. Conjunto de dados disponíveis para 20 anos (ao que parece).
4. Os dados estão organizados por seções (1: 'leis', 2: 'servidores', e 3: 'contratos, acordos')
e meses: https://dados.gov.br/dataset/diario-oficial-da-uniao-materias-publicadas-em-2021
5. Vamos partir da página da organização de interesse e verificar os *links* existentes.

In [ ]:
import requests
import urllib.request
from bs4 import BeautifulSoup as BS

---
1. Este é o código base. Poderíamos fazê-lo como função (avoid repetition).
Todavia, para fins didáticos, vamos repeti-lo algumas vezes.

    1. `url` de interesse
    2. `import urllib`
    3. resposta = `urllib.request.urlopen(url)`
    4. Use BeautifulSoup (from bs4) para ler a resposta.

In [ ]:
url_base = 'https://dados.gov.br/'
url_in = 'https://dados.gov.br/organization/imprensa-nacional-in'
response = urllib.request.urlopen(url_in)
html_in = BS(response)
html_in

2. A resposta é um objeto python do tipo BeautifulSoup
    1. Ou "*Beautiful Soup parses the given HTML document into a tree of Python objects*"
    2. Desse modo, funciona como um intérprete do texto propriamente dito.
3. Você pode simplesmente transformar tudo em texto com a opção `.text`
    1. Isso seria útil se você domina Regular Expressions, por exemplo `import re`

In [ ]:
html_in.text

4. No caso de objeto BeautifulSoup, você pode também acessar diretamente os headings: `h1, h2, h3, ...` Um a um.
5. **Headings compõem a linguagem HTML**
6. HTML é similar a XML, como vimos. Com tags do tipo: <h1>Título H1</h1>
7. Que, como observam, também funciona em Markdown (jupyter).

In [ ]:
html_in.h2

8. Você pode fazer um *loop* em todos os heading tipo h2

In [ ]:
for heads2 in html_in.find_all('h2'):
    print(heads2)

9. A resposta da `BeautifulSoup` contém ainda tags, seções (`div`), paragraphs (`p`)
    1. Veja a lista completa [aqui](https://developer.mozilla.org/en-US/docs/Web/HTML/Element)
10. Também é possível utilizar a função `find_all()` para buscas específicas.
11. Nesse caso, vamos buscar todos os "objetos" do tipo 'a' que contêm um atributo **href**.

In [ ]:
links1 = html_in.find_all('a', href=True)
links1

12. Note que o comando `find_all()` retorna uma lista que dentro contêm objetos BS.

13. Vamos trabalhar agora com a lista para **EXTRAIR** a informação referente ao link, o **href**.
14. Usamos a função `get`: `objeto.get('href')`
15. Nesse caso específico, constatamos que o diretório de interesse é `dataset`.
Usamos uma condicional, dentro de uma *list comprehension*

In [ ]:
links2 = [l.get('href') for l in links1 if 'dataset' in l.get('href')]
links2

## Here is where we automate it!
<marquee style='width: 30%; color: red; font-size:30px'><b>Uau!</b></marquee>
<p style="font-size:50px">&#128526;</p>

16. Para raspar todos os *links*, possivelmente, restringiríamos essa lista,
excluindo o primeiro e os 4 últimos links **[1:-4]**
17. Adicionalmente, há links repetidos, vamos usar `list(set))`, para eliminar links repetidos

In [ ]:
links2_full = list(set(links2[1:-4]))
links2_full

18. A título de exemplo, vamos escolher somente um arquivo para dar o download.
19. Lembrem-se que cada página contém
    * um ano,
    * depois temos links para os meses,
    * cada mês contém os dias,
    * cada dia três seções e
    * cada seção, varíos tipos de ATOS.

<marquee style='width: 30%; color: orange; font-size:30px'><b>Ufa!</b></marquee>

20. Agora que já temos os links dos anos. Vamos **repassar o processo**, com a página de um ano específico.

In [ ]:
url_in_2021 = f'{url_base}{links2[2]}'
response = urllib.request.urlopen(url_in_2021)
html_in_2021 = BS(response)
html_in_2021

21. Vamos novamente localizar os links **href**

In [ ]:
links3 = html_in_2021.find_all('a', href=True)
links3

22. Conseguimos localizar que os links para os arquivos dos dias, vamos em busca dos links para os arquivos **zip**
23. Identificamos a pasta `resource` de interesse. Vamos filtrar os links usando novamente *list comprehension*

In [ ]:
links4 = [l.get('href') for l in links3 if 'resource' in l.get('href')]
links4

24. Novamente, vamos escolher somente uma página para investigar.

In [ ]:
page = f'{url_base}{links4[0]}'
page

25. Mesmo processo novamente.

In [ ]:
response = urllib.request.urlopen(page)
page_soup = BS(response)
page_soup

26. Examinando o retorno, conseguimos identificar uma **tag** do tipo `'p'` que contém o link para o arquivo **zip**.

In [ ]:
my_first_data = page_soup.find_all('p', {'class': 'muted ellipsis'})
my_first_data

27. A lista contém só um item [0]. Vamos extrair o objeto `'href'`

In [ ]:
my_first_data = my_first_data[0].find('a')['href']
my_first_data

28. Do próprio nome do link, extraímos o nome do arquivo. `S{section}{mes}{ano}`.

In [ ]:
name = my_first_data.split('/')
name = [n for n in name if 'zip' in n][0]
name

29. Pronto. Salvemos o arquivo em formato zip:  `'wb'` (write bytes).
30. Note o uso do `requests.get()`, no lugar de `urllib.request.urlopen()` que vinhámos usando.

In [ ]:
r = requests.get(my_first_data)
with open(f'data/{name}', 'wb') as handler:
    handler.write(r.content)


---
# Outro exemplo de captura por meio de *DataLakes*

## https://basedosdados.org/dataset/mundo-kaggle-olimpiadas
---

1. É necessário instalar basedosdados

## **Caution**
Aqui, e na recomendação da página *basedosdados*, realizamos a instalação via **pip**

Under the hood -- i.e., sem nos avisar, o **pip** desinstalou o `pandas 1.3` corrente e trocou pela versão `1.22`.

Não me lembro, mas alguma função que eu tinha utilizado recentemente só estava disponível na versão `1.3`.

**IRL** -- in real life -- eu só uso `conda envs` no `PyCharm`, com isso consigo controlar quais versões de quais bibliotecas estão instaladas.

*Jupyter* é ótimo para aulas e apresentações, mas não é funcional para *scripts* e programas completos!

2. Base dos Dados possui

# 504

organizações

In [96]:
# !pip install basedosdados

3. Além de instalar a biblioteca *basedosdados*, para realizar buscas ou ter acesso às bases,
é necessário ter uma conta gratuita (limitada a 10 projetos no gcloud).
4. Você pode se cadastrar de acordo com essas [instruções](https://basedosdados.github.io/mais/access_data_local/).
5. Com isso, e o login na conta do Google (gmail), você terá como preencher corretamente o parâmetro:
`billing_project_id=<nome_do_seu_projeto_cadastrado_no_gcloud>`

In [97]:
import basedosdados as bd
# Para carregar o dado direto no pandas
df = bd.read_table(dataset_id='mundo_kaggle_olimpiadas',
                   table_id='microdados',
                   billing_project_id='idp-class')

Downloading: 100%|██████████| 269718/269718 [00:46<00:00, 5851.29rows/s]


6. Vamos dar uma primeira olhada na base que lemos, direto para uma DataFrame do pandas.

In [ ]:
df.info()

# Tratamento dados

Feita a **Captura**, nos seus vários formatos, é necessário compreender, interpretar, analisar os dados, sejam eles:

1. Numéricos
2. Ou textuais
3. Ou mistos